# 143. Movie Review Sentiment Classification

출처 : Jay Alammar http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/



<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-sentence-classification.png" width="500"/>

이 노트에서는 사전 훈련 된 딥 러닝 모델을 사용하여 일부 텍스트를 처리합니다. 그런 다음 해당 모델의 출력을 사용하여 텍스트를 분류합니다. 텍스트는 영화 리뷰의 문장 목록입니다. 

## Models: Sentence Sentiment Classification
우리의 목표는 (데이터 세트의 것과 같은) 문장을 가져 와서 1 (문장이 긍정적 인 느낌을 나타냄) 또는 0 (문장이 부정적인 감정을 나타냄)을 생성하는 모델을 만드는 것입니다. 우리는 이것을 다음과 같이 생각할 수 있습니다 :

<img src="https://jalammar.github.io/images/distilBERT/sentiment-classifier-1.png" width="500"/>

내부적으로 모델은 실제로 두 가지 모델로 구성됩니다.

*DistilBERT는 문장을 처리하고 추출 된 정보를 다음 모델로 전달합니다. DistilBERT는 HuggingFace 팀이 개발하고 공개 한 BERT의 작은 버전입니다. 가볍고 빠른 BERT 버전으로 성능과 거의 일치합니다.

* 다음 모델 인 scikit learn의 기본 로지스틱 회귀 모델은 DistilBERT의 처리 결과를 받아 문장을 positive 또는 negative (각각 1 또는 0)로 분류합니다.

두 모델간에 전달되는 데이터는 크기가 768 인 벡터입니다.이 벡터를 분류에 사용할 수있는 문장에 대한 embedding 으로 생각할 수 있습니다.


<img src="https://jalammar.github.io/images/distilBERT/distilbert-bert-sentiment-classifier.png" width="500"/>

## Dataset
이 예에서 사용할 데이터 세트는 [SST2] (https://nlp.stanford.edu/sentiment/index.html)이며, 영화 리뷰의 문장이 포함되어 있으며 각 문장은 positive (값 1) 또는 negative 로 label 되어 있습니다. (값이 0 임) 

<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600">
    sentence
    </th>
    <th class="mdc-text-purple-600">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      apparently reassembled from the cutting room floor of any given daytime soap
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      they presume their audience won't sit still for a sociology lesson
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      this is a visually stunning rumination on love , memory , history and the war between art and commerce
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      jonathan parker 's bartleby should have been the be all end all of the modern office anomie films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
</table>


In [0]:
!pip install transformers

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import torch
from transformers import *

## Importing the dataset

In [0]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [4]:
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


For performance reasons, we'll only use 2,000 sentences from the dataset

In [0]:
df = df[:2000]

몇개의 sentence 가 "positive" (value 1) 이고 몇개가  "negative" (having the value 0) 인지 count

In [6]:
df[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

## Loading the Pre-trained BERT model

In [7]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = DistilBertModel, DistilBertTokenizer, 'distilbert-base-uncased'

## BERT instead of distilBERT
#model_class, tokenizer_class, pretrained_weights = BertModel, BertTokenizer, 'bert-base-uncased'

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

현재 'model' 변수는 사전 훈련 된 distilBERT 모델을 보유하고 있습니다. BERT 버전은 더 작지만 훨씬 빠르며 훨씬 적은 메모리를 필요로합니다.

## Model #1: Preparing the Dataset
문장을 BERT에 전달하려면 먼저 필요한 형식으로 문장을 처리하기 위해 최소한의 처리가 필요합니다.

### Tokenization
첫 번째 단계는 문장을 토큰화하는 것입니다. BERT가 원하는 형식으로 word 와 subword 로 나눕니다.

In [0]:
tokenized = df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" width="700"/>

### Padding

토큰 화 후, 각 문장은 토큰 list 로 표시됩니다. BERT 가 example 을 one batch 로 한 번에 처리하도록 하기 위해 padding 에 의해 같은 길이의 list 로 만들어야 하고 이것을 2-d array 로 표시합니다.

In [9]:
tokenized.values

array([list([101, 1037, 18385, 1010, 6057, 1998, 2633, 18276, 2128, 16603, 1997, 5053, 1998, 1996, 6841, 1998, 5687, 5469, 3152, 102]),
       list([101, 4593, 2128, 27241, 23931, 2013, 1996, 6276, 2282, 2723, 1997, 2151, 2445, 12217, 7815, 102]),
       list([101, 2027, 3653, 23545, 2037, 4378, 24185, 1050, 1005, 1056, 4133, 2145, 2005, 1037, 11507, 10800, 1010, 2174, 14036, 2135, 3591, 1010, 2061, 2027, 19817, 4140, 2041, 1996, 7511, 2671, 4349, 3787, 1997, 11829, 7168, 9219, 1998, 28971, 2308, 1999, 8301, 8737, 2100, 4253, 102]),
       ...,
       list([101, 2023, 2028, 8704, 2005, 1996, 11848, 1998, 7644, 1037, 3622, 2718, 102]),
       list([101, 1999, 1996, 2171, 1997, 2019, 9382, 18988, 1998, 4089, 3006, 3085, 17312, 1010, 1996, 3750, 1005, 1055, 2252, 4332, 1037, 6397, 3239, 2000, 1996, 2200, 2381, 2009, 9811, 2015, 2000, 6570, 102]),
       list([101, 1996, 3185, 2003, 25757, 2011, 1037, 24466, 16134, 2008, 1005, 1055, 2074, 6388, 2438, 2000, 7344, 3686, 1996, 7731, 4378, 209

In [11]:
max_len = max([len(i) for i in tokenized.values])
max_len

59

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded = pad_sequences(tokenized.values, padding='post')
padded

array([[  101,  1037, 18385, ...,     0,     0,     0],
       [  101,  4593,  2128, ...,     0,     0,     0],
       [  101,  2027,  3653, ...,     0,     0,     0],
       ...,
       [  101,  2023,  2028, ...,     0,     0,     0],
       [  101,  1999,  1996, ...,     0,     0,     0],
       [  101,  1996,  3185, ...,     0,     0,     0]], dtype=int32)

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [20]:
padded.shape

(2000, 59)

### Masking
패딩된 sequence 를 BERT에 직접 보내면 약간 혼동됩니다. 입력을 처리 할 때 추가 한 패딩을 무시 (마스크)하도록 다른 변수를 만들어야 하는데, 이 것을 attention_mask 라 합니다.

In [22]:
attention_mask = np.where(padded !=0, 1, 0 )
attention_mask.shape

(2000, 59)

## Model #1: And Now, Deep Learning!

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tutorial-sentence-embedding.png" width="700"/>

`model ()`함수는 BERT를 통해 문장을 실행합니다. 처리 결과는`last_hidden_states`로 반환됩니다.

In [24]:
%%time
input_ids = torch.tensor(padded).to(torch.int64)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


CPU times: user 3min 1s, sys: 2.81 s, total: 3min 4s
Wall time: 1min 33s


필요한 출력 부분 만 슬라이스합시다. 그것은 각 문장의 첫 번째 토큰에 해당하는 출력입니다. BERT가 문장 분류를하는 방식은 모든 문장의 시작 부분에`[CLS]`(분류 용)라는 토큰을 추가하는 것입니다. 해당 토큰에 해당하는 출력은 전체 문장에 대한 임베딩으로 간주 될 수 있습니다.

<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />

로지스틱 회귀 모델의 feature 로 사용되므로`features` 변수에 저장합니다.

In [32]:
last_hidden_states[0].shape

torch.Size([2000, 59, 768])

In [0]:
features = last_hidden_states[0][:,0,:].numpy()

The labels indicating which sentence is positive and negative now go into the `labels` variable

In [0]:
labels = df[1]

## Model #2: Train/Test Split
이제 데이터 세트를 훈련 세트와 테스트 세트로 나누겠습니다 (SST2 훈련 세트에서 2,000 문장만 사용하더라도)

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

<img src="http://jalammar.github.io/images/distilBERT/bert-distilbert-train-test-split-sentence-embedding.png" width="800"/>

We now train the LogisticRegression model. 

In [42]:
lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

<img src="http://jalammar.github.io/images/distilBERT/bert-training-logistic-regression.png" width="800"/>

## Evaluating Model #

In [43]:
lr_clf.score(test_features, test_labels)

0.844

## Proper SST2 scores
For reference, the [highest accuracy score](http://nlpprogress.com/english/sentiment_analysis.html) for this dataset is currently **96.8**. DistilBERT can be trained to improve its score on this task – a process called **fine-tuning** which updates BERT’s weights to make it achieve a better performance in this sentence classification task (which we can call the downstream task). The fine-tuned DistilBERT turns out to achieve an accuracy score of **90.7**. The full size BERT model achieves **94.9**.